In [ ]:
import pandas as pd  

# Load CSV
csv_path = r"C:\Users\Admin\OneDrive\Documents\USIU\Datawarehousing\EndSemExam\DSA-2040_Practical_Exam_Whitney-Wairimu-Gituara-528\Data_Warehousing\ETL\Online Retail.csv"  
data = pd.read_csv(csv_path, encoding='latin1')  

# Preview of first few rows
print("Preview of dataset:")
print(data.head())  

# Convertion of InvoiceDate to datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')  
# errors='coerce' replaces bad dates with NaT

# Count missing values before cleaning
print("\nMissing values count:")
print(data.isna().sum())  

# Drop rows where essential columns are missing
data = data.dropna(subset=['InvoiceNo', 'StockCode', 'Quantity', 'UnitPrice', 'InvoiceDate'])  

# Fill missing CustomerID with 'Unknown'
data['CustomerID'] = data['CustomerID'].fillna('Unknown')
# Fill missing Description with 'No Description'
data['Description'] = data['Description'].fillna('No Description')

# Summary after cleaning
print("\nMissing values after cleaning:")
print(data.isna().sum())  
print("\nRows remaining after extraction and cleaning:", len(data))

# Transformation Steps
# This includes removing outliers and creating new calculated fields
# Remove outliers: drop rows where Quantity is negative or UnitPrice is zero or negative
data = data[(data['Quantity'] >= 0) & (data['UnitPrice'] > 0)]

# Calculate TotalSales
data['TotalSales'] = data['Quantity'] * data['UnitPrice']

# Extract Month, Quarter, and Year for time analysis
data['Month'] = data['InvoiceDate'].dt.month
data['Quarter'] = data['InvoiceDate'].dt.quarter
data['Year'] = data['InvoiceDate'].dt.year

# Create a customer summary table
customer_summary = data.groupby('CustomerID').agg({
    'TotalSales': 'sum',
    'InvoiceNo': 'count',
    'Country': 'first'
}).rename(columns={
    'InvoiceNo': 'TotalPurchases'
}).reset_index()

# Filter for sales in the last year (from Aug 12, 2025)
end_date = pd.to_datetime('2025-08-12')
start_date = end_date - pd.DateOffset(years=1)
recent_sales = data[(data['InvoiceDate'] >= start_date) & (data['InvoiceDate'] <= end_date)]

# Sort recent sales by InvoiceDate
recent_sales = recent_sales.sort_values('InvoiceDate')

# Preview the transformed datasets
print("Recent sales preview:")
print(recent_sales.head())

print("\nCustomer summary preview:")
print(customer_summary.head())

# Load section
# This section includes saving the transformed data to new CSV files
import sqlite3 #importing sqlite3 module

# Create/connect to a SQLite database file
conn = sqlite3.connect("retail_dw.db")
cursor = conn.cursor()

# -------------------------
# Creating Dimension Tables
# -------------------------

# Customer Dimension
cursor.execute("""
CREATE TABLE IF NOT EXISTS CustomerDim (
    CustomerID TEXT PRIMARY KEY,
    Country TEXT
)
""")

# Product Dimension
cursor.execute("""
CREATE TABLE IF NOT EXISTS ProductDim (
    StockCode TEXT PRIMARY KEY,
    Description TEXT
)
""")

# Time Dimension
cursor.execute("""
CREATE TABLE IF NOT EXISTS TimeDim (
    TimeID INTEGER PRIMARY KEY AUTOINCREMENT,
    InvoiceDate DATE,
    Year INTEGER,
    Quarter INTEGER,
    Month INTEGER
)
""")

# Country Dimension
cursor.execute("""
CREATE TABLE IF NOT EXISTS CountryDim (
    Country TEXT PRIMARY KEY
)
""")

# -------------------------
# Fact Table creation

cursor.execute("""
CREATE TABLE IF NOT EXISTS SalesFact (
    InvoiceNo TEXT,
    CustomerID TEXT,
    StockCode TEXT,
    InvoiceDate DATE,
    Quantity INTEGER,
    UnitPrice REAL,
    TotalSales REAL,
    FOREIGN KEY (CustomerID) REFERENCES CustomerDim(CustomerID),
    FOREIGN KEY (StockCode) REFERENCES ProductDim(StockCode),
    FOREIGN KEY (InvoiceDate) REFERENCES TimeDim(InvoiceDate)
)
""")

# ---------------------------------------------------
# Inserting data into Dimension Tables

# Customer Dimension
customer_data = data[['CustomerID', 'Country']].drop_duplicates()
customer_data.to_sql('CustomerDim', conn, if_exists='replace', index=False)

# Product Dimension
product_data = data[['StockCode', 'Description']].drop_duplicates()
product_data.to_sql('ProductDim', conn, if_exists='replace', index=False)

# Time Dimension
time_data = data[['InvoiceDate', 'Year', 'Quarter', 'Month']].drop_duplicates()
time_data.to_sql('TimeDim', conn, if_exists='replace', index=False)

# Country Dimension
country_data = data[['Country']].drop_duplicates()
country_data.to_sql('CountryDim', conn, if_exists='replace', index=False)

# Inserting data into the Fact Table
fact_data = data[['InvoiceNo', 'CustomerID', 'StockCode', 'InvoiceDate', 'Quantity', 'UnitPrice', 'TotalSales']]
fact_data.to_sql('SalesFact', conn, if_exists='replace', index=False)

# Commit and close connection
conn.commit()
conn.close()

print("Data loaded into retail_dw.db successfully!")

# Function that performs the full ETL process and logs the number of rows processed at each stage.
def full_etl_process(csv_path, db_name="retail_dw.db"):
    """ 
    Performs the full ETL process: Extract, Transform, Load.
    Logs the number of rows at each stage.
    
    Parameters:
        csv_path (str): Path to the CSV file
        db_name (str): Name of the SQLite database file to create
    """
    try:
        # -------------------------
        # Extract
        # -------------------------
        print("=== Extract Stage ===")
        # Read CSV into a pandas DataFrame
        # This is the first step to bring raw data into Python for processing
        data = pd.read_csv(csv_path, encoding='latin1')
        print("Rows read from CSV:", len(data))
        
        # Convert InvoiceDate to datetime
        # This allows us to easily filter, sort, and extract time attributes
        data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')
        
        # Drop rows missing essential values to avoid errors in calculations or database load
        data = data.dropna(subset=['InvoiceNo', 'StockCode', 'Quantity', 'UnitPrice', 'InvoiceDate'])
        print("Rows after dropping essential missing values:", len(data))
        
        # Fill missing CustomerID with 'Unknown' to avoid null values in dimension table
        # This ensures every transaction is linked to a customer
        data['CustomerID'] = data['CustomerID'].fillna('Unknown')
        print("Rows after filling missing CustomerID:", len(data))
        
        # -------------------------
        # Transform
        # -------------------------
        print("\n=== Transform Stage ===")
        
        # Remove outliers: negative Quantity or non-positive UnitPrice
        # Outliers could distort totals and summaries, so we filter them out
        data = data[(data['Quantity'] >= 0) & (data['UnitPrice'] > 0)]
        print("Rows after removing outliers:", len(data))
        
        # Calculate TotalSales for each transaction
        # This will be used in the fact table for sales analysis
        data['TotalSales'] = data['Quantity'] * data['UnitPrice']
        
        # Extract Month, Quarter, and Year for time dimension
        # These are used to analyze sales over different periods
        data['Month'] = data['InvoiceDate'].dt.month
        data['Quarter'] = data['InvoiceDate'].dt.quarter
        data['Year'] = data['InvoiceDate'].dt.year
        
        # Create customer summary table for CustomerDim
        # Aggregates total sales, total purchases, and keeps country info
        customer_summary = data.groupby('CustomerID').agg({
            'TotalSales': 'sum',
            'InvoiceNo': 'count',
            'Country': 'first'
        }).rename(columns={'InvoiceNo': 'TotalPurchases'}).reset_index()
        print("Customer summary rows:", len(customer_summary))
        
        # Filter recent sales (Aug 12, 2024 → Aug 12, 2025)
        # This creates a subset for analyzing last year’s performance
        end_date = pd.to_datetime('2025-08-12')
        start_date = end_date - pd.DateOffset(years=1)
        recent_sales = data[(data['InvoiceDate'] >= start_date) & (data['InvoiceDate'] <= end_date)]
        print("Recent sales rows (last year):", len(recent_sales))
        
        # -------------------------
        # Load
        # -------------------------
        print("\n=== Load Stage ===")
        try:
            # Connect to SQLite database
            # Using a database allows us to store structured data for reporting and analysis
            conn = sqlite3.connect(db_name)
            
            # Load dimension tables first
            # Dimensions contain descriptive data used to categorize and filter facts
            customer_summary.to_sql('CustomerDim', conn, if_exists='replace', index=False)
            
            # Product dimension: unique products with description
            product_data = data[['StockCode', 'Description']].drop_duplicates()
            product_data.to_sql('ProductDim', conn, if_exists='replace', index=False)
            
            # Time dimension: unique invoice dates with extracted Year, Quarter, Month
            time_data = data[['InvoiceDate', 'Year', 'Quarter', 'Month']].drop_duplicates()
            time_data.to_sql('TimeDim', conn, if_exists='replace', index=False)
            
            # Country dimension: unique countries
            country_data = data[['Country']].drop_duplicates()
            country_data.to_sql('CountryDim', conn, if_exists='replace', index=False)
            
            # Load fact table after dimensions
            # Facts contain measurable data (sales, quantities) and link to dimensions via keys
            fact_data = data[['InvoiceNo', 'CustomerID', 'StockCode', 'InvoiceDate', 'Quantity', 'UnitPrice', 'TotalSales','Country']]
            fact_data.to_sql('SalesFact', conn, if_exists='replace', index=False)
            
            conn.commit()
            print("Data loaded into database:", db_name)
            print("Fact table rows:", len(fact_data))
        except sqlite3.Error as e:
            print("Database error:", e)
        finally:
            conn.close()
            
    except FileNotFoundError:
        print(f"Error: CSV file not found at path: {csv_path}")
    except pd.errors.ParserError:
        print("Error: Could not parse CSV file. Check file format.")
    except Exception as e:
        print("Unexpected error:", e)



Preview of dataset:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  

Missing values count:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country         

: 